In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
ames = pd.read_csv('data/ames_housing.csv')

In [ ]:
ames.head()

In [ ]:
c1 = corr.abs().unstack()
c1[c1!=1].sort_values(ascending = False)

In [ ]:
plt.figure()
plt.hist(ames.SalePrice)


# It looks like there may be a few houses that are major outliers.  Based on the correllation matrix above, it looks like OverallQual and GrLivArea are the most highly correlated with Sale Price, so I will investigate there.

In [ ]:
plt.figure()
plt.scatter(ames.OverallQual, ames.SalePrice)

In [ ]:
plt.figure()
plt.scatter(ames.GrLivArea, ames.SalePrice)

In [ ]:
ames.SaleCondition.value_counts(dropna=False)

## It looks like there are 4 really odd houses where the relationship between GrLivArea and SalesPrice break down, so I'm planning to drop those from the model.  I realize I could also use the Log of sales price, but it seems like the vast majority of datapoints fit the non-log scale, so I'm making the judgment to just drop the weird looking data points.

## This is a huge dataset, and I'm assuming you aren't asking us to make the best possible model, so below, I've pulled in enough features that I think are relevant and useful to a linear model.  I've also decided to simply drop nulls here since I want to see the overall improvement in the model by filling certain nulls with appropriate fills.

In [ ]:
ames_orig_X = ames[ames.GrLivArea<4000][['LotFrontage', 'LotArea', 'OverallQual', 'TotalBsmtSF', 'SalePrice',
                    'GrLivArea', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'PoolArea', 'YrSold'
                   ]].dropna()
ames_orig_y = ames_orig_X.SalePrice
ames_orig_X = ames_orig_X.drop('SalePrice', axis=1)

In [ ]:
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(ames_orig_X, ames_orig_y)
lr.fit(X_train, y_train)
orig_pred = lr.predict(X_test)
print('RMSE is {:.4f}'.format(np.sqrt(mean_squared_error(y_test, orig_pred))))
print('R-squared is {:.4f}'.format(lr.score(X_test, y_test)))

##  Without any feature engineering or filling, the model seems to be OK.  I tested my hypothesis above and using log of sales price without first filtering by GrLivArea makes a big improvement, but once I filtered by GrLivArea< 4000, the improvement became somewhat negligible. 

In [ ]:
ames['PoolArea'].value_counts()

# Now I'll fill in nulls for some of the categorical values that I plan to use in my model (or as components of engineered features).  For categoricals that I'm encoding as ordinals, I'll just fill the NaNs as zero. 

In [ ]:
ames = pd.read_csv('data/ames_housing.csv')

In [3]:
ames['PoolQC'] = ames['PoolQC'].fillna('None')
ames['Alley'] = ames['Alley'].fillna("None")
ames['FireplaceQu'] = ames['FireplaceQu'].fillna("None")
ames['MiscFeature'] = ames['MiscFeature'].fillna("None")

In [4]:
ames.FireplaceQu.value_counts(dropna=False)

None    690
Gd      380
TA      313
Fa       33
Ex       24
Po       20
Name: FireplaceQu, dtype: int64

In [5]:
ames = ames.replace({"BsmtCond": {None:0, "No": 0, "Po": 1, "Fa": 2, "TA": 3, "Gd": 4, "Ex": 5}})
ames = ames.replace({"BsmtQual" : {None: 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
ames = ames.replace({"GarageQual" : {None: 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
ames = ames.replace({"GarageCond" : {None: 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
ames = ames.replace({"PoolQC" : {"None" : 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})
ames = ames.replace({"FireplaceQu" : {"None" : 0, "No" : 0, "Po" : 1, "Fa" : 2, "TA": 3, "Gd" : 4, "Ex" : 5}})

In [6]:
ames[['BsmtCond', 'BsmtQual', 'GarageQual', 'GarageCond', 'PoolQC', 'FireplaceQu']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
BsmtCond       1460 non-null int64
BsmtQual       1460 non-null int64
GarageQual     1460 non-null int64
GarageCond     1460 non-null int64
PoolQC         1460 non-null int64
FireplaceQu    1460 non-null int64
dtypes: int64(6)
memory usage: 68.5 KB


# Below are some (hopefully useful) engineered features.  There are plenty more I could imagine making, but I gather that the point of this assignment is merely to show that I understand the idea behind engineering features, which I think the below probably does.  

In [7]:
ames['TotalSF'] = ames.TotalBsmtSF + ames['1stFlrSF'] + ames['2ndFlrSF']
ames['AboveGrRooms'] = ames.KitchenAbvGr + ames.BedroomAbvGr + ames.FullBath + ames.HalfBath
ames['AgeSold'] = ames.YrSold - ames.YearRemodAdd #documentation says that Remodel age is same as YrBuilt if no remodel.
ames['MSSubClass'] = ames['MSSubClass'].astype(str)
ames['LotFrontage'] = ames['LotFrontage'].fillna(ames.LotFrontage.median()) #don't want to mess up distribution
ames['LotMetric'] = ames['LotFrontage'] * ames['LotArea']
ames['GarageOverall'] = ames['GarageCond'] * ames['GarageArea'] * ames['GarageCars'] * ames['GarageQual']
ames['PoolOverall'] = ames['PoolArea'] * ames['PoolQC']
ames['BasementOverall'] = ames['BsmtCond'] * ames['BsmtQual']
ames['FireplaceOverall'] = ames.FireplaceQu * ames.Fireplaces

In [8]:
ames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 89 columns):
Id                  1460 non-null int64
MSSubClass          1460 non-null object
MSZoning            1460 non-null object
LotFrontage         1460 non-null float64
LotArea             1460 non-null int64
Street              1460 non-null object
Alley               1460 non-null object
LotShape            1460 non-null object
LandContour         1460 non-null object
Utilities           1460 non-null object
LotConfig           1460 non-null object
LandSlope           1460 non-null object
Neighborhood        1460 non-null object
Condition1          1460 non-null object
Condition2          1460 non-null object
BldgType            1460 non-null object
HouseStyle          1460 non-null object
OverallQual         1460 non-null int64
OverallCond         1460 non-null int64
YearBuilt           1460 non-null int64
YearRemodAdd        1460 non-null int64
RoofStyle           1460 non-null obj

##  Now I've added a few columns and created dummies.  Conceivably, I could create new engineered features with dummies (e.g., Heating (dummies) * HeatingQC (encoded ordinal)), but I could see the number of columns getting out of hand rather quickly. 

In [16]:
ames_improved = ames[ames['GrLivArea']<4000][['SalePrice', 'TotalSF','AboveGrRooms', 'AgeSold', 'MSSubClass', 'MSZoning', 'LotMetric',
                      'Alley', 'GarageOverall', 'PoolOverall', 'BasementOverall', 'FireplaceOverall',
                      'CentralAir', 'OverallQual', 'OverallCond', 'GrLivArea', 'TotRmsAbvGrd'
                     ]]

In [19]:
ames_improved.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1459
Data columns (total 17 columns):
SalePrice           1456 non-null int64
TotalSF             1456 non-null int64
AboveGrRooms        1456 non-null int64
AgeSold             1456 non-null int64
MSSubClass          1456 non-null object
MSZoning            1456 non-null object
LotMetric           1456 non-null float64
Alley               1456 non-null object
GarageOverall       1456 non-null int64
PoolOverall         1456 non-null int64
BasementOverall     1456 non-null int64
FireplaceOverall    1456 non-null int64
CentralAir          1456 non-null object
OverallQual         1456 non-null int64
OverallCond         1456 non-null int64
GrLivArea           1456 non-null int64
TotRmsAbvGrd        1456 non-null int64
dtypes: float64(1), int64(12), object(4)
memory usage: 204.8+ KB


In [20]:
ames_improved = pd.get_dummies(ames_improved, drop_first=True)

In [21]:
ames_improved.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1456 entries, 0 to 1459
Data columns (total 34 columns):
SalePrice           1456 non-null int64
TotalSF             1456 non-null int64
AboveGrRooms        1456 non-null int64
AgeSold             1456 non-null int64
LotMetric           1456 non-null float64
GarageOverall       1456 non-null int64
PoolOverall         1456 non-null int64
BasementOverall     1456 non-null int64
FireplaceOverall    1456 non-null int64
OverallQual         1456 non-null int64
OverallCond         1456 non-null int64
GrLivArea           1456 non-null int64
TotRmsAbvGrd        1456 non-null int64
MSSubClass_160      1456 non-null uint8
MSSubClass_180      1456 non-null uint8
MSSubClass_190      1456 non-null uint8
MSSubClass_20       1456 non-null uint8
MSSubClass_30       1456 non-null uint8
MSSubClass_40       1456 non-null uint8
MSSubClass_45       1456 non-null uint8
MSSubClass_50       1456 non-null uint8
MSSubClass_60       1456 non-null uint8
MSSubClass_

# Now it's time to compare the old model to the new one!

In [13]:
ames_orig_X = ames[ames.GrLivArea<4000][['LotFrontage', 'LotArea', 'OverallQual', 'TotalBsmtSF', 'SalePrice',
                    'GrLivArea', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'GarageArea', 'PoolArea', 'YrSold'
                   ]].dropna()
ames_orig_y = ames_orig_X.SalePrice
ames_orig_X = ames_orig_X.drop('SalePrice', axis=1)
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(ames_orig_X, ames_orig_y)
lr.fit(X_train, y_train)
orig_pred = lr.predict(X_test)
print('RMSE of Original Model is {:.4f}'.format(np.sqrt(mean_squared_error(y_test, orig_pred))))
print('R-squared of Original Model is {:.4f}'.format(lr.score(X_test, y_test)))

RMSE of Original Model is 33319.1825
R-squared of Original Model is 0.8146


In [14]:
lr = LinearRegression()
X_train, X_test, y_train, y_test = train_test_split(ames_improved.drop('SalePrice', axis=1), ames_improved.SalePrice)
lr.fit(X_train, y_train)
pred = lr.predict(X_test)
print('RMSE of Spiffy New Model is {:.4f}'.format(np.sqrt(mean_squared_error(y_test, pred))))
print('R-squared of Spiffy New Model is {:.4f}'.format(lr.score(X_test, y_test)))

RMSE of Spiffy New Model is 31040.5008
R-squared of Spiffy New Model is 0.8339


# Adding some features seems to have improved the model somewhat.  Now it's time to see if scaling, using other models, and/or polynomial features can improve it more.

# Also, after playing around a bit, it seems that with scaled X values, transforming sale price to the log of sale price makes more of a difference.  I must confess I don't understand *why* that's the case.

In [77]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import Ridge, Lasso, ElasticNet
scaler = MinMaxScaler()
X_train, X_test, y_train, y_test = train_test_split(ames_improved.drop('SalePrice', axis=1), np.log(ames_improved.SalePrice))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [78]:
linear_scaled = LinearRegression()
ridge_ames = Ridge(alpha=1)
lasso_ames = Lasso()
eNet_ames = ElasticNet()

In [79]:
linear_scaled.fit(X_train_scaled, y_train)
predictions = linear_scaled.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = linear_scaled.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : 0.8564 
The RMSE value is 0.1489


In [80]:
ridge_ames.fit(X_train_scaled, y_train)
predictions = ridge_ames.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = ridge_ames.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : 0.8780 
The RMSE value is 0.1372


In [81]:
lasso_ames.fit(X_train_scaled, y_train)
predictions = lasso_ames.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = lasso_ames.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : -0.0013 
The RMSE value is 0.3932


In [82]:
eNet_ames.fit(X_train_scaled, y_train)
predictions = eNet_ames.predict(X_test_scaled)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = eNet_ames.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : -0.0013 
The RMSE value is 0.3932


In [49]:
lasso_ames.coef_

array([ 0.13320915,  0.        , -0.00281001,  0.        ,  0.02255429,
       -0.        ,  0.        ,  0.        ,  0.11682661,  0.        ,
        0.        ,  0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        , -0.        , -0.        , -0.        ,
        0.        , -0.        , -0.        ,  0.        ,  0.        ,
       -0.        ,  0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        ])

In [50]:
ridge_ames.coef_

array([ 0.10504906, -0.00733893, -0.04039272,  0.02003258,  0.06282615,
       -0.00773938,  0.03189313,  0.02815681,  0.09190231,  0.02883661,
        0.06898049, -0.00658998, -0.02543209, -0.0061238 , -0.01115786,
       -0.01231133, -0.02725767, -0.00169547, -0.00326354, -0.02230555,
       -0.00211388, -0.01320186, -0.00809402, -0.00483169, -0.00239059,
       -0.01130244,  0.08046072,  0.03360815,  0.13882841,  0.08708537,
        0.01230284,  0.00760962,  0.02051971])

##  So the act of scaling seems to have significantly improved the model, and the Ridge seems to have improved a little too, but I'm not really sure what's going on with the Lasso and Elastic Nets.  

## For now, I'm going to just try and see what happens when I add polynomial features using a pipeline.

In [106]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline 
X_train, X_test, y_train, y_test = train_test_split(ames_improved.drop('SalePrice', axis=1), np.log(ames_improved.SalePrice))


In [107]:
lm_pipe = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
lm_pipe.fit(X_train, y_train)
predictions = lm_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = lm_pipe.score(X_test, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : 0.0436 
The RMSE value is 0.3762


In [108]:
ridge_pipe = make_pipeline(PolynomialFeatures(degree=2), Ridge())
ridge_pipe.fit(X_train, y_train)
predictions = ridge_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = ridge_pipe.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : -14.3847 
The RMSE value is 0.2560


C:\ProgramData\Anaconda3\lib\site-packages\scipy\linalg\basic.py:223: RuntimeWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number: 1.934251277291456e-19
  ' condition number: {}'.format(rcond), RuntimeWarning)


In [111]:
lasso_pipe = make_pipeline(PolynomialFeatures(degree=2), Lasso(alpha=1)) #I played around here with a bunch of alphas.
lasso_pipe.fit(X_train, y_train)
predictions = lasso_pipe.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
score = lasso_pipe.score(X_test_scaled, y_test)
print('The r2 value is : {:.4f}'.format(score), '\nThe RMSE value is {:.4f}'.format(rmse))

The r2 value is : -7.0033 
The RMSE value is 0.1829


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


#  I'm guessing here that I'm misinterpreting what [model].score() is returning, since it seems like the "scores" of different models vary widely even though the RMSE seems to be improving.